## Лабораторная работа "Введение в ML"

В этой лабораторной вы:

- познакомитесь с базовыми библиотеками для работы с табличными данными — `numpy` и `pandas`
- поближе посмотрите на простейшие задачи машинного обучения: классификацию и регрессию
- попробуете несколько метрик и поймёте, почему выбор метрики это важно
- обучите несколько простых моделей
- увидите связь между сложностью модели и переобучением
- убедитесь, что без данных всё тлен

Загрузка самых базовых библиотек:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

### [NumPy](https://numpy.org/doc/stable/user/index.html)

С 1995 numeric, с 2006 NumPy — «Numerical Python extensions» или просто «NumPy»

Возможности библиотеки NumPy:
* работать с многомерными массивами (таблицами)
* быстро вычислять математические функций на многомерных массивах

Ядро пакета NumPy — объект [ndarray](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html)

**Важные отличия** между NumPy arrays и Python sequences:
* NumPy array имеет фиксированную длину, которая определяется в момент его создания (в отличие от Python lists, которые могут расти динамически)
* Элементы в NumPy array должны быть одного типа
* Можно выполнять операции непосредственно над NumPy arrays

**Скорость** NumPy достигается с помощью:
* реализации на C
* векторизации и броадкастинга (broadcasting). Например, произведение массивов совместимых форм.

Теперь давайте разберёмся подробнее и сделаем что-нибудь приятное и полезное в `numpy`!

### Индексация

В NumPy работает привычная индексация Python, ура! Включая использование отрицательных индексов и срезов (slices)

<div class="alert alert-info">
<b>Замечание 1:</b> Индексы и срезы в многомерных массивах не нужно разделять квадратными скобками,
т.е. вместо <b>matrix[i][j]</b> нужно использовать <b>matrix[i, j]</b>. Первое тоже работает, но сначала выдаёт строку i, потом элемент j в ней.
</div>

<div class="alert alert-danger">
<b>Замечание 2:</b> Срезы в NumPy создают view, а не копии, как в случае срезов встроенных последовательностей Python (string, tuple and list).
</div>

In [2]:
ones_matrix = np.ones((5, 5))
ones_submatrix_view = ones_matrix[::2,::2] # creates a view, not copy
ones_matrix[::2,::2] = np.zeros((3, 3))
ones_submatrix_view

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

### Ссылка на Яндекс.Контест

Решения и ответы в задачах, расположенных ниже, загружайте в контест на автоматическую проверку:
https://new.contest.yandex.ru/60376/start


**1.** Реализуйте функцию, принимающую на вход два одномерных массива `first_array` и `second_array` и возвращающую матрицу, в которой первый массив соответствует первому столбцу матрицы, второй — второму.

Вероятно первое, что приходит вам на ум, это конкатенация и транспонирование:

In [3]:
def construct_matrix(first_array, second_array):
    """
    Construct matrix from pair of arrays
    :param first_array: first array
    :param second_array: second array
    :return: constructed matrix
    """
    return np.column_stack([first_array, second_array]) # <- your first right code here

In [4]:
construct_matrix(np.array([1,2]),np.array([3,4]))

array([[1, 3],
       [2, 4]])

(в скобках заметим, что конкатенировать можно vertically, horizontally, depth wise методами vstack, hstack, dstack по трём осям (0, 1 и 2, соотвественно), либо в общем случае `np.concatenate` — поиграйтесь ниже с прекрасным примером четырёхмерной точки, чтобы точно всё для себя понять)

In [5]:
p = np.arange(1).reshape([1, 1, 1, 1])
p

array([[[[0]]]])

In [6]:
print("vstack: ", np.vstack((p, p)).shape)
print("hstack: ", np.hstack((p, p)).shape)
print("dstack: ", np.dstack((p, p)).shape)

vstack:  (2, 1, 1, 1)
hstack:  (1, 2, 1, 1)
dstack:  (1, 1, 2, 1)


In [7]:
np.concatenate((p, p), axis=3).shape

(1, 1, 1, 2)

Но, поскольку операция транспонирования [делает массив non-contiguous](https://numpy.org/doc/stable/user/basics.copies.html#other-operations), мы в этой задаче **запретим** ей пользоваться и порекомедуем воспользоваться, например, методом [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html).

**2.** Реализуйте функцию, принимающую на вход массив целых неотрицательных чисел `nums` и возвращающую самый частый элемент массива.

In [8]:
def most_frequent(nums):
    """
    Find the most frequent value in an array
    :param nums: array of ints
    :return: the most frequent value
    """
    unique, counts = np.unique(nums, return_counts=True)
    max_id = np.argmax(counts)
    return nums[max_id]

### Переходим к работе с данными

Прежде всего, загрузим данные и сделаем из них красивые pandas-таблички. Они взяты из параллели RecSys соревнования https://yandex.ru/cup/ml/. Но мы будем иметь дело не со всеми данными, а только с их частью. Данные у нас будут про заведения общественного питания (больше бюрократический терминологии!)

Файлы с данными можно найти [здесь](https://disk.yandex.ru/d/YWvCNRQMb7QSQA).

Задачей будет **предсказание среднего чека** (average_bill) по некоторым другим свойствам заведения.

In [9]:
base = '/content/sample_data/'

In [10]:
data = pd.read_csv(base + 'organisations.csv')
features = pd.read_csv(base + 'features.csv')
rubrics = pd.read_csv(base + 'rubrics.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/organisations.csv'

В основном мы будем работать с табличкой `data`; остальное вам может пригодиться, если вы захотите знать, какое содержание стоит за кодами признаков.

## Изучение данных

Посмотрите на данные. В этом вам поможет метод ``head`` pandas-таблички.

In [ ]:
# <Your code here>
print(data.head())

Полезно посмотреть внимательнее на то, с какими признаками нам предстоит работать.

* **org_id** вам не понадобится;
* **city** - город, в котором находится заведение (``msk`` или ``spb``);
* **average_bill** - средний чек в заведении - он будет нашим таргетом;
* **rating** - рейтинг заведения;
* **rubrics_id** - тип заведения (или несколько типов). Соответствие кодов каким-то человекочитаемым типам живёт в табличке ``rubrics``
* **features_id** - набор неких фичей заведения. Соответствие кодов каким-то человекочитаемым типам живёт в табличке ``features``

Обратите внимание, что **rubrics_id** и **features_id** - это не списки, а разделённые пробелами строки. Когда вам захочется работать с отдельными фичами из мешка фичей для данного заведения, вам придётся всё-таки превратить их в списки (здесь поможет метод `split` для строк).

Чтобы быстро восстанавливать по рубрикам и фичам их нормальные названия, сделайте словари вида ``код_фичи:название_фичи``

In [ ]:
# <Your code here>
rubric_dict = rubrics.set_index('rubric_id')['rubric_name'].to_dict()

Посмотрим, какими бывают типы заведений:

In [ ]:
rubric_dict

Мы что-то поняли про признаки, которыми нам предстоит пользоваться. Теперь время посмотреть на таргет. Вооружившись функциями ``hist`` и ``scatter`` из библиотеки ``matplotlib``, а также методом ``isna`` для pandas-таблиц разберитесь, какие значения принимают таргеты, есть ли там там выбросы, пропуски или ещё какие-то проблемы.

&nbsp;

<details>
  <summary>Когда будете готовы, кликните сюда, чтобы посмотреть ответ</summary>
    <ol>
      <li>Среди таргетов довольно много пропусков;</li>
      <li>Все таргеты - это числа, кратные 500;</li>
      <li>Есть какие-то адские значения, превышающие 100 000 (видимо, выбросы);</li>
      <li>В целом, число ресторанов с данным средним чеком быстро падает с ростом среднего чека. Для средних чеков, больших 2500, заведений уже совсем мало. Примерно у 2/3 заведений средний чек 500.</li>
    </ol>
</details>

In [ ]:
print(data['average_bill'].isna().sum())

**Базовая очистка данных**

Раз есть треш, давайте чистить данные.

С пропусками можно бороться по-разному (даже и с пропусками в таргете), но пока мы сделаем самую простую вещь: дропнем все заведения, для которых мы не знаем средний чек.

Уберите из них все заведения, у которых средний чек неизвестен или превышает 2500. Пока есть опасение, что их слишком мало, чтобы мы смогли обучить на них что-нибудь.

**3. Введите в Контест количество заведений, которое у вас получилось после очистки**.

Дальше мы будем работать с очищенными данными.

In [ ]:
# <Your code here>
filtered_data = data.dropna(subset=['average_bill'])
filtered_data = filtered_data[filtered_data['average_bill'] <= 2500]
print(len(filtered_data))

In [ ]:
print(filtered_data.head())

**4. Посчитайте и введите в Контест разность между средними арифметическими average_bill в кафе Москвы и Санкт-Петербурга. Округлите ответ до целого.**

&nbsp;

<details>
  <summary>Небольшая подсказка</summary>
  Примените часто используемый метод groupby.
</details>

In [ ]:
# <Your code here>
cafe_data = filtered_data[filtered_data['rubrics_id'].str.contains('30774', na=False)]
grouped_data = cafe_data.groupby('city')['average_bill'].mean()
msk_avg = grouped_data.get('msk', 0)
spb_avg = grouped_data.get('spb', 0)
difference = round(msk_avg - spb_avg)
print(difference)

Давайте ещё немного поизучаем данные. Ответьте на вопросы:

1. Есть ли разница между средними чеками в Москве и Санкт-Петербурге?
2. Коррелирует ли средний чек с рейтингом?
3. Есть ли разница в среднем чеке между ресторанами и пабами (см. соответствующие типы из ``rubrics``)?

&nbsp;

<details>
  <summary>Когда будете готовы, кликните сюда, чтобы посмотреть ответ</summary>
    <ol>
      <li>В целом, да. Вы могли бы сравнить средние (в Москве больше) или медианы (они равны, потому что уж больно много где средний чек 500). Этого, конечно, мало для того, чтобы сделать вывод. Нужно проверять какие-то статические критерии, которые изучаются в курсе по статистике. Не будем останавливаться на этом подробно. Поскольку данные совсем не нормальные, никакой t-тест не сработает; мы бы предложили использовать критерий Манна-Уитни (см. википедию и функцию mannwhitneyu из библиотеки scipy.stats).</li>
      <li>Какая-то корреляция между ними есть но уж больно неубедительная (рекомендуем построим на одном графике boxplot рейтинга по каждому значению среднего чека для визуализации). Конечно, дна становится меньше с ростом среднего чека, но, видимо, в предсказании это особо не используешь;</li>
      <li>Несомненно, в ресторанах средний чек выше. Это и невооружённым глазом видно, и с помощью критерия Манна-Уитни можно проверить.</li>
    </ol>
</details>

## Формулируем задачу

Прежде, чем решать задачу, её надо сформулировать.

**Вопрос первый**: это классификация или регрессия? Подумайте над этим.

&nbsp;

<details>
  <summary>Когда будете готовы, кликните сюда, чтобы посмотреть ответ</summary>
    Ответ не столь однозначен, как хотелось бы. С одной стороны, таргет принимает всего четыре значения, и потому это может быть классификацией с 4 классами. С другой стороны, таргеты - это не абстрактные "треугольник", "круг", "квадрат", а вещественные числа, и когда мы вместо 500 предсказываем 2500, это явно хуже, чем вместо 1500 предсказать 2000. В целом, задачу можно решать и так, и так; мы будем смотреть на метрики обеих задач.
</details>

**Вопрос второй**: какие метрики мы будем использовать для оценки качества решения? Какие метрики вы предложили бы для этой задачи как для задачи классификации? А для этой задачи, как для задачи регрессии?

&nbsp;

<details>
  <summary>Когда будете готовы, кликните сюда, чтобы посмотреть ответ</summary>
    
    Начнём с классификации. Метрика accuracy не очень хороша из-за несбалансированности классов. Действительно, классификатор, который всегда говорит 500, будет иметь accuracy примерно 0.66, хотя это никак не отражает практическую ценность модели. Как мы увидим, самая большая проблема будет заключаться в том, чтобы научиться выделять заведения с большими чеками, а их меньше всего и в accuracy они вносят самый маленький вклад. Есть разные способы с этим бороться, один -- использовать sklearn.metrics.balanced_accuracy_score. Его идея, грубо говоря, в том, чтобы по каждому классу найти, какая доля объектов этого класса правильно классифицирована, а потом эти доли усреднить. Тогда у бессмысленного классификатора, который всем ставит 500, будет скор 1/5 (ведь классов 5), а чтобы получить прежние 2/3, нужно будет научиться в каждом классе правильно ставить хотя бы 2/3 меток.    
    
    Теперь что касается регрессии. Основых метрики две - MSE и MAE. Из первой стоит извлекать корень, чтобы получать интерпретируемые человеком значения, а вторая менее агрессивна к выбросам (впрочем, выбросов тут уже нет, мы их все выкинули). Без дополнительной информации не очень понятно, какую выбирать, можно брать любую. А выбирать надо: ведь даже банальные модели "предсказывай всегда среднее" и "предсказывай всегда медиану" будут по-разному ранжироваться этими метриками.
    
</details>

**Вопрос третий**: а не взять ли нам какую-нибудь более экзотическую метрику? Например, MAPE (определение в учебнике в главе про оценку качества моделей). А как вам такое соображение: допустим, заказчик говорит, что пользователи будут расстраиваться, только если мы завысили средний чек - так давайте поправим MSE или MAE, обнуляя те слагаемые, для которых предсказанный таргет меньше истинного. Вот это хорошая метрика или нет?

&nbsp;

<details>
  <summary>Когда будете готовы, кликните сюда, чтобы посмотреть ответ</summary>
    
    Что касается MAPE, у нас нет тех проблем, с которой она борется. Вот если бы у нас были средние чеки от 500 до миллиона, мы бы столкнулись с ситуацией, что большие ошибки для больших чеков доминировали бы в сумме для MSE и MAE (500 вместо 1000 меркнет по сравнению с 500к вместо миллиона). Говоря поэтически, мы бы оптимизировали модель для миллионеров, забыв про простых трудяг. И было бы логично перейти от парадигмы "ошибаемся на 500 рублей" к парадигме "ошибаемся на 50%". Но у нас все таргеты примерно одного порядка, MAPE нам особо ни к чему.
    
    Вторая метрика коварна тем, что её можно "накрутить" безо всякой пользы для дела. А именно, модель, которая всегда предсказывает средний чек в миллион, была бы идеальна. Но все бы расстраивались и не ходили есть. Другое дело, что можно ввести разные веса для ошибок в большую и в меньшую сторону, но опять же - пока нет показаний к тому, что это нужно.
    
</details>

## Применяем ML

Теперь время разбить данные на обучающую и тестовую выборку. Делается это с помощью функции ``train_test_split`` из пакета ``sklearn``. При этом очень важно сделать две вещи:

* Зафиксировать ``random_state=42`` (да, именно этот, а то ваши модели могут не зайти в Контест), чтобы всё, что мы делаем, было воспроизводимо (иначе от перезапуска к перезапуску числа могут меняться, и мы не будем понимать, из-за чего это происходит).
* Сделать стратификацию по таргету. В противном случае у нас в трейне и тесте могут оказаться разные пропорции классов (обычно особенно страдают мало представленные классы), что неутешительно скажется на результате.

**Обратите внимание**, что если вы побьёте выборку на train и test по-другому, ваши результаты могут не зайти в контест.

In [ ]:
clean_data_train, clean_data_test = train_test_split(
    filtered_data, stratify=filtered_data['average_bill'], test_size=0.33, random_state=42)

Теперь нам нужен **бейзлайн** - очень простая модель, с которой мы в дальнейшем будем сравниваться.

Поскольку мы ещё не знаем никаких умных классов моделей, все модели мы будем писать руками. А именно, мы напишем две простых модели на основе ``sklearn.baseRegressorMixin`` и ``sklearn.base.ClassifierMixin`` (посмотрите примеры в документации sklearn и сделайте так же):

* Модель для задачи регрессии, которая для всех заведений предсказывает одно число — среднее значение среднего чека;
* Модель для задачи классификации, которая для всех заведений предсказывает один класс — самый частый класс (ироничным образом он в данном случае совпадает с медианой).

**Важно!** Мы будем много раз повторять вам мантру о том, что **информация из тестовой выборки не должна протекать в процесс обучения**. Так вот, и среднее, и самый частый класс вы должны считать именно на обучающей выборке!

**5 и 6. Напишите эти две модели и сдайте в Контест**. В процессе проверки модели будут и обучаться, и предсказывать.

Заметим, что для этих моделей нам вообще не нужны какие-то "фичи"; мы работаем только с таргетом.

У каждой модели есть (как минимум) два метода: `fit` (обучает модель по фичам `X` и таргету `y`) `predict` (предсказывает по фичам `X`)

In [ ]:
from scipy.stats import mode
from sklearn.base import RegressorMixin, ClassifierMixin
import numpy as np

class MeanRegressor(RegressorMixin):
    # Predicts the mean of y_train
    def fit(self, X=None, y=None):
        '''
        Parameters
        ----------
        X : array like, shape = (n_samples, n_features)
        Training data features
        y : array like, shape = (_samples,)
        Training data targets
        '''
        self.mean_ = np.mean(y)
        return self

    def predict(self, X=None):
        '''
        Parameters
        ----------
        X : array like, shape = (n_samples, n_features)
        Data to predict
        '''
        n_samples = X.shape[0] if hasattr(X, 'shape') else len(X)
        return np.full(n_samples, self.mean_)

from sklearn.base import ClassifierMixin
import numpy as np
from scipy.stats import mode

class MostFrequentClassifier(ClassifierMixin):
    # Predicts the rounded (just in case) median of y_train
    def fit(self, X=None, y=None):
        '''
        Parameters
        ----------
        X : array like, shape = (n_samples, n_features)
        Training data features
        y : array like, shape = (_samples,)
        Training data targets
        '''
        if len(y) == 0:
            self.most_frequent_ = None
        else:
            mode_result = mode(y)
            # Check if the mode result is a scalar or an array
            if np.isscalar(mode_result.mode):
                self.most_frequent_ = mode_result.mode
            elif mode_result.mode.size > 0: # Check if the mode result array is not empty
                self.most_frequent_ = mode_result.mode[0]
            else:
                # Handle the case where mode doesn't return a mode (e.g., all elements are unique)
                # In this case, we can just take the first element as a fallback or handle as appropriate
                self.most_frequent_ = y.iloc[0] if hasattr(y, 'iloc') else y[0]

        return self

    def predict(self, X=None):
        '''
        Parameters
        ----------
        X : array like, shape = (n_samples, n_features)
        Data to predict
        '''
        n_samples = X.shape[0] if hasattr(X, 'shape') else len(X)
        if self.most_frequent_ is None:
            return np.empty(n_samples) # Or handle as appropriate for no frequent value
        else:
            return np.full(n_samples, self.most_frequent_)

Обучим наши модели

In [ ]:
reg = MeanRegressor()
reg.fit(y=clean_data_train['average_bill'])
reg_pred = reg.predict(X=clean_data_test)

clf = MostFrequentClassifier()
clf.fit(y=clean_data_train['average_bill'])
clf_pred = clf.predict(X=clean_data_test)

In [ ]:
from sklearn.metrics import mean_squared_error, balanced_accuracy_score
import numpy as np

# Calculate RMSE for the regressor
reg_rmse = np.sqrt(mean_squared_error(clean_data_test['average_bill'], reg_pred))
print(f"MeanRegressor RMSE: {reg_rmse}")

# Calculate RMSE and balanced accuracy for the classifier
clf_rmse = np.sqrt(mean_squared_error(clean_data_test['average_bill'], clf_pred))
clf_balanced_accuracy = balanced_accuracy_score(clean_data_test['average_bill'], clf_pred)

print(f"MostFrequentClassifier RMSE: {clf_rmse}")
print(f"MostFrequentClassifier Balanced Accuracy: {clf_balanced_accuracy}")

Обучите модели и оцените их качество на тестовой выборке. В качестве метрик возьмём RMSE (``np.sqrt`` от ``sklearn.metrics.mean_squared_error``) и ``sklearn.metrics.balanced_accuracy_score``.

Для регрессионной модели имеет смысл считать только RMSE (значения будут не кратны 500, точно мы угадывать не будем никогда), а вот для классификационной можно найти обе метрики. Сделайте это. Какая модель оказалась лучше по RMSE?

<details>
  <summary>Когда будете готовы, кликните сюда</summary>
    
  Казалось бы, регрессор никогда не угадывает, но он в каком-то смысле лучше классификатора - справедливо ли это? Возможно. Несуществующий пользователь модели вряд ли будет задавать вопросы "почему средний чек не кратен 500?" Ну, выдали около 800 - ок, понятно.
    
</details>

## Усложнение модели

Бейзлайны будут нашей отправной точкой. Строя дальнейшие модели, мы будем спрашивать себя: получилось ли лучше бейзлайна? Если нет или если не особо, то в чём смысл усложнения?

Начнём с использования фичи ``city``. Мы уже видели, что в разных городах и средние чеки разные. Легко проверить, что *медиана* средних чеков всё же одна и та же и в Москве, и в Санкт-Петербурге (ох уж этот вездесущий средний чек 500!), поэтому с классификатором мы ничего не сделаем. Но вот регрессор можно попробовать починить.

**7. Напишите регрессор, для каждого заведения предсказывающий среднее значение в том же городе (на обучающей выборке, конечно) и сдайте его в Контест**. Вам может помочь то, что булевы `pandas` и `numpy` столбцы можно умножать на численные — в такой ситуации False работает, как ноль, а True как единица.

In [ ]:
from sklearn.base import RegressorMixin
import pandas as pd
import numpy as np

class CityMeanRegressor(RegressorMixin):
    def fit(self, X, y):
        '''
        Parameters
        ----------
        X : pandas DataFrame, shape = (n_samples, n_features)
        Training data features (must include 'city' column)
        y : pandas Series, shape = (_samples,)
        Training data targets
        '''
        if 'city' not in X.columns:
            raise ValueError("Input DataFrame X must contain a 'city' column.")

        train_data = X.copy()
        train_data['average_bill'] = y
        self.city_means_ = train_data.groupby('city')['average_bill'].mean().to_dict()
        self.global_mean_ = np.mean(y) # Fallback for cities not seen during training
        return self

    def predict(self, X):
        '''
        Parameters
        ----------
        X : pandas DataFrame, shape = (n_samples, n_features)
        Data to predict (must include 'city' column)
        '''
        if 'city' not in X.columns:
            raise ValueError("Input DataFrame X must contain a 'city' column.")

        predictions = X['city'].apply(lambda city: self.city_means_.get(city, self.global_mean_))
        return predictions

Обучите регрессор и сравните его по метрике RMSE с бейзлайнами. Получилось ли улучшить метрику?

Лучше стало, но, правда, не очень сильно. В этот момент очень важно не просто радовать руководителя приростом в третьем знаке, но и думать о том, что происходит.

Средний средний чек по Москве равен 793, в Санкт-Петербурге - 676, а в целом - 752 рубля. MSE, увы, не поможет вам ответить на вопрос, стало ли лучше пользователю, если вы ему вместо 752 рублей назвали 793. Здесь вскрывается весьма существенный порок MSE в этой задаче. Дело в том, что наш изначальный таргет делит заведения на некоторые "ценовые категории", и различие в средних чеках 500 и 1000 в самом деле существенно. Наверное, мы хотели бы как раз правильно предсказывать ценовые категории. Но MSE не очень помогает нам об этом судить. Дальше мы ещё подумаем, как это исправить.

В любом случае, несмотря на улучшение метрики, мы пока не можем судить, стало ли по жизни лучше от усложнения модели.

Поручинившись немного, возьмём на вооружение другую идею. Давайте использовать типы заведений!

Но с типами есть некоторая проблема: в столбце ``rubrics_id`` не всегда один идентификатор, часто их несколько, и всего комбинаций довольно много. Чтобы не возиться с малочисленными типами, давайте сольём их в один безликий ``other``.

Итак, добавьте в обучающие и тестовые данные столбец ``modified_rubrics``, в котором будет то же, что и в ``rubrics_id``, если соответствующая комбинация рубрик содержит хотя бы 100 заведений из обучающей (!) выборки, и строка ``other`` в противном случае.

Здесь вам поможет контейнер ``Counter`` из библиотеки ``collections``.

In [ ]:
from collections import Counter

# Count occurrences of each rubrics_id in the training data
rubric_counts = Counter(clean_data_train['rubrics_id'])

# Identify rubrics_id combinations that appear less than 100 times
rare_rubrics = {rubric for rubric, count in rubric_counts.items() if count < 100}

# Create the modified_rubrics column in the training data
clean_data_train['modified_rubrics'] = clean_data_train['rubrics_id'].apply(
    lambda x: 'other' if x in rare_rubrics else x
)

# Create the modified_rubrics column in the testing data
clean_data_test['modified_rubrics'] = clean_data_test['rubrics_id'].apply(
    lambda x: 'other' if x in rare_rubrics else x
)

Теперь настало время написать могучий классификатор, который по заведению предсказывает медиану средних чеков среди тех в обучающей выборке, у которых с ним одинаковые `modified_rubrics` и город (вы спросите, почему медиану, а не самый частый -- спишем это на вдохновение; самый частый тоже можно брать - но медиана работает лучше).

**8. Напишите классификатор и сдайте в Контест**.

In [ ]:
# your code

Сравните обученный классификатор по метрикам RMSE и balanced_accuracy_score с нашими бейзлайнами. Получилось ли улучшить?

Обратите внимание что рост accuracy по сравнению с бейзлайном при этом на порядок меньше:

accuracy_score

Predict most frequent:  0.6947666195190948

Predict by rubric and city:  0.7095709570957096

Для диагностики напечатайте для каждого класса тестовой выборки, сколько в нём объектов и скольким из них наш классификатор приписал правильный класс. Что вы видите?

&nbsp;

<details>
  <summary>Когда будете готовы, кликните сюда, чтобы посмотреть ответ</summary>
    
  Вы, вероятно, видите то, что мы стали однозначно лучше по сравнению с бейзлайном детектировать средний чек 1000 и 1500 (хотя всё равно не очень хорошо + ценой ухудшения качества на среднем чеке 500), а вот чеки 2000 и 2500 нам ну никак не даются.
    
</details>

**Кстати**. А вы понимаете, почему приведённый выше пайплайн классификации был не очень удачным с точки зрения архитектуры? Почему его было бы правильнее воплотить по-другому?

&nbsp;

<details>
  <summary>Когда будете готовы, кликните сюда, чтобы посмотреть ответ</summary>
Собственно говоря, и не было никакого пайплайна. К счастью, у нас была одна обучающая выборка, мы на ней посчитали список рубрик для modified_rubrics и радовались жизни. Но если бы нам надо было переобучать всё на новых данных, пришлось бы помнить, что их надо везде пересчитать (ведь у нас могли появиться новые рубрики с хотя бы 100 представителями). А уж никакую кросс-валидацию (кто знает - тот поймёт) с нашим подходом к делу и вовсе бы не получилось сделать без боли.
    
Поэтому в следующей лабораторной вы научитесь делать честные пайплайны, в которых преобразование данных, генерация фичей и обучение классификатора будут объединены в один понятный процесс, происходящий на этапе fit.
</details>

## Слишком простые и слишком сложные модели

Бейзлайны у нас слишком просты и потому не очень полезны в жизни. Но если сложность модели растёт бесконтрольно, то тоже получается плохо.

Давайте рассмотрим конкретный пример. Создадим классификатор, использующий одновременно `rubrics_id` и `features_id`.

Сделайте следующее:

- для каждого объекта обучающей выборки сконкатенируйте строку `rubrics_id` с разделителем (например, буквой 'q') и содержимым `features_id`. Полученный столбец озаглавьте `modified_features`. Это не самый клёвый способ заиспользовать все фичи, но сейчас пока сойдёт. Причём на сей раз не будем выкидывать мало представленные значения (вся информация важна, не так ли?).
- при этом для тестовой выборке заменяйте на строку `other` все конкатенации, которые не встретились в обучающей выборке.

То есть элементы в этом столбце будут иметь вид `other` или `30776 30774 q 3502045032 11741 3502045016 1046...`.

Теперь обучите классификатор, который для заведения предсказывает медиану среднего чека по всем объектам тестовой выборки с таким же, как у него, значением `modified_features`, а если такого в обучающей выборке нет, то глобальную медиану среднего чека по всей обучающей выборке.

**9. Загрузите в Контест предсказания этого классификатора на тестовой выборке**

Мы ждём файла **.csv**, у которого в каждой строке будет только одно число - предсказание классификатора.

Возможно, вам будет полезна библиотека ``tqdm``, позволяющая отслеживать в реальном времени, сколько времени уже крутится цикл и сколько итераций ещё осталось. Впрочем, если вы всё написали нормально, то должно работать не очень долго.

Модель, очевидно, очень сложная. Число параметров (различных категорий) в ней сопоставимо с числом объектов в обучающей выборке. А получилось ли хорошо?

Давайте посчитаем RMSE и balanced_accuracy_score на обучающей и на тестовой выборках.

**10. Введите их в Контест**

Налицо переобучение: на трейне метрики отличные, на тесте - вообще никакие

В общем, не гонитесь за чрезмерной сложностью модели..

In [ ]:
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.base import ClassifierMixin

# Create the modified_features column for training data
clean_data_train['modified_features'] = clean_data_train['rubrics_id'] + 'q' + clean_data_train['features_id']

# Get unique modified_features from training data
train_features = set(clean_data_train['modified_features'].unique())

# Create the modified_features column for testing data, replacing unseen with 'other'
clean_data_test['modified_features'] = clean_data_test.apply(
    lambda row: row['rubrics_id'] + 'q' + row['features_id'] if (row['rubrics_id'] + 'q' + row['features_id']) in train_features else 'other',
    axis=1
)

class ModifiedFeaturesMedianClassifier(ClassifierMixin):
    def fit(self, X, y):
        '''
        Parameters
        ----------
        X : pandas DataFrame, shape = (n_samples, n_features)
        Training data features (must include 'modified_features' column)
        y : pandas Series, shape = (_samples,)
        Training data targets
        '''
        if 'modified_features' not in X.columns:
            raise ValueError("Input DataFrame X must contain a 'modified_features' column.")

        train_data = X.copy()
        train_data['average_bill'] = y
        self.feature_medians_ = train_data.groupby('modified_features')['average_bill'].median().to_dict()
        self.global_median_ = np.median(y) # Fallback for unseen features
        return self

    def predict(self, X):
        '''
        Parameters
        ----------
        X : pandas DataFrame, shape = (n_samples, n_features)
        Data to predict (must include 'modified_features' column)
        '''
        if 'modified_features' not in X.columns:
            raise ValueError("Input DataFrame X must contain a 'modified_features' column.")

        if X['modified_features'].empty:
            print("Warning: 'modified_features' column in prediction data is empty.")
            return pd.Series([], dtype=float) # Return an empty Series

        predictions = X['modified_features'].apply(lambda feature: self.feature_medians_.get(feature, self.global_median_))
        return predictions

# Train the classifier
mf_clf = ModifiedFeaturesMedianClassifier()
mf_clf.fit(X=clean_data_train, y=clean_data_train['average_bill'])

# Make predictions on the test set
mf_clf_pred = mf_clf.predict(X=clean_data_test)

# Add print statements to check the predictions before saving
print(f"Shape of predictions Series: {mf_clf_pred.shape}")
print("Predictions head:")
print(mf_clf_pred.head())

# Format predictions as "index,prediction" strings
formatted_predictions = [f"{index},{prediction}" for index, prediction in mf_clf_pred.items()]

# Save formatted predictions to a text file
with open('modified_features.txt', 'w') as f:
    for line in formatted_predictions:
        f.write(line + '\n')

print("Predictions saved to modified_features.txt")

In [ ]:
# Read from the text file and parse index and prediction
data_from_txt = []
with open('modified_features.txt', 'r') as f:
    for line in f:
        index, prediction = line.strip().split(',')
        data_from_txt.append({'index': int(index), 'average_bill': float(prediction)})

# Convert to DataFrame and save to CSV
predictions_df = pd.DataFrame(data_from_txt)
# Set the 'index' column as the DataFrame index
predictions_df = predictions_df.set_index('index')
predictions_df.to_csv('modified_features.csv', index=True, header=False)

print("Predictions saved to modified_features.csv")

In [ ]:
from sklearn.metrics import mean_squared_error, balanced_accuracy_score
import numpy as np

# Make predictions on the training set
mf_clf_train_pred = mf_clf.predict(X=clean_data_train)

# Calculate metrics on the training set
mf_clf_train_rmse = np.sqrt(mean_squared_error(clean_data_train['average_bill'], mf_clf_train_pred))
mf_clf_train_balanced_accuracy = balanced_accuracy_score(clean_data_train['average_bill'], mf_clf_train_pred)

# Calculate metrics on the test set (predictions are already available in mf_clf_pred)
mf_clf_test_rmse = np.sqrt(mean_squared_error(clean_data_test['average_bill'], mf_clf_pred))
mf_clf_test_balanced_accuracy = balanced_accuracy_score(clean_data_test['average_bill'], mf_clf_pred)

# Print the results rounded to two decimal places
print(f"{mf_clf_train_rmse:.2f} {mf_clf_train_balanced_accuracy:.2f} {mf_clf_test_rmse:.2f} {mf_clf_test_balanced_accuracy:.2f}")

## ML без данных что компутер без электричества

Возможно, вы смотрите на полученные выше результаты и думаете: вот если бы мы не какие-то убогие медианы предсказывали, а гоняли бы нейросети, то тут-то бы всё и получилось!

Но, увы, совсем даже не всегда от счастья нас отделяет выбор хорошей модели (и стратегии обучения). Если данные не очень, то даже самая крутая модель не сработает. В этой ситуации нужно либо добывать новые фичи каким-то образом, либо собирать новые данные (увеличивать датасет), либо просто бросать задачу.

Давайте посмотрим, что выжмет из наших данных одна из самых мощных моделей для табличных данных - градиентный бустинг на решающих деревьях в исполнении [CatBoost](https://catboost.ai/).

Но прежде, чем сделать fit, нам надо облагородить данные. Несмотря на то, что CatBoost отлично работает с категориальными фичами, мешок признаков из `rubrics_id` или `features_id` может ему оказаться не по зубам. Поэтому мы соберём датасет в пристойную матрицу, создав для каждого типа рубрик и фичей отдельный столбец и записав там единицы для тех объектов, у которых эта рубрика или фича имеет место.

В матрице почти все элементы будут нулями. Такие матрицы считаются **разреженными** и их можно хранить гораздо эффективней, чем просто таблицей. Этим и займёмся)

Есть несколько форматов хранения разреженных матриц (многие из них реализованы в [пакете sparse библиотеки scipy](https://docs.scipy.org/doc/scipy/reference/sparse.html)), и каждый пригоден для чего-то своего.

Создавать разреженную матрицу лучше в [формате COO](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_array.html#scipy.sparse.coo_array). Он предполагает, что разреженная матрица задаётся в виде трёх списков: `row`, `col`, `data`, причём каждая тройка `(row[i], col[i], data[i])` кодирует элемент со значением `data[i]`, стоящий на позиции `(row[i], col[i])`. Считается, что на позициях `(row, col)`, которые ни разу не встретились, стоят нули.

Нетрудно видеть, что заполнять такую матрицу - одно удовольствие, и особенно этому помогает тот факт, что **пара `(row, col)` может встретиться несколько раз** (тогда в итоговой матрице на соответствующей позиции стоит сумма соответствующих `data[i]`). Но, с другой стороны, почти ничего другого с такой матрицей не сделаешь: произвольного доступа к элементам она не предоставляет, умножить её тоже особо ничего не умножишь. Поэтому для дальнейшего использования созданную таким образом матрицу преобразуют в один из более удобных форматов, например, [CSR (compressed sparse row)](https://scipy-lectures.org/advanced/scipy_sparse/csr_matrix.html). Он, к примеру, хорошо подходит для умножения на вектор (потому что матрица хранится по строкам). Не будем разбирать его подробно, но можете почитать по ссылке, если интересно.

Вам нужно будет превратить обучающие и тестовые данные в разреженные матрицы `sparse_data_train` и `sparse_data_test` соответственно, таким образом, что:

- столбец `city` превратится в столбец из единиц и нулей (например, 1 - Москва, 0 - Питер);
- столбец `rating` перекочует в разреженные матрицы без изменений;
- каждый типы рубрик и каждая фича превратятся в отдельный 0-1-принак;

В тестовой выборке будут фичи, которых в обучающей выборке не было. С ними можно по-разному работать, но давайте создадим дополнительную фантомную фичу `feature_other`, в которой будет то, сколько неизвестных по обучающей выборке фичей есть у данного объекта.

In [ ]:
from scipy.sparse import coo_matrix, hstack
import numpy as np

def create_sparse_features(df, all_rubrics, all_features):
    """
    Creates sparse features from a DataFrame including city, rating, rubrics, and features_id.

    Args:
        df: Pandas DataFrame with 'city', 'rating', 'rubrics_id', and 'features_id' columns.
        all_rubrics: A list of all unique rubric IDs from the training data.
        all_features: A list of all unique feature IDs from the training data.

    Returns:
        A sparse matrix containing the features.
    """
    n_samples = len(df)
    row_indices = []
    col_indices = []
    data_values = []

    # City feature (1 for msk, 0 for spb)
    city_feature = (df['city'] == 'msk').astype(int).values
    row_indices.extend(range(n_samples))
    col_indices.extend([0] * n_samples)
    data_values.extend(city_feature)

    # Rating feature
    # Handle potential NaN values in rating by replacing with a placeholder or mean/median
    # For now, we'll fill NaN with 0, but a more sophisticated approach might be needed
    rating_feature = df['rating'].fillna(0).values
    row_indices.extend(range(n_samples))
    col_indices.extend([1] * n_samples)
    data_values.extend(rating_feature)

    # Rubrics features
    rubric_col_start = 2  # Start index for rubric features
    rubric_id_to_col = {rubric_id: i + rubric_col_start for i, rubric_id in enumerate(all_rubrics)}

    for i, rubrics_list in enumerate(df['rubrics_id'].str.split().fillna('')):
        for rubric_id in rubrics_list:
            if rubric_id in rubric_id_to_col:
                row_indices.append(i)
                col_indices.append(rubric_id_to_col[rubric_id])
                data_values.append(1)

    # Features_id features
    features_col_start = rubric_col_start + len(all_rubrics)
    feature_id_to_col = {feature_id: i + features_col_start for i, feature_id in enumerate(all_features)}
    feature_other_col = features_col_start + len(all_features) # Column for 'feature_other'

    for i, features_list in enumerate(df['features_id'].str.split().fillna('')):
        unknown_feature_count = 0
        for feature_id in features_list:
            if feature_id in feature_id_to_col:
                row_indices.append(i)
                col_indices.append(feature_id_to_col[feature_id])
                data_values.append(1)
            else:
                unknown_feature_count += 1
        # Add 'feature_other' count
        if unknown_feature_count > 0:
             row_indices.append(i)
             col_indices.append(feature_other_col)
             data_values.append(unknown_feature_count)


    # Create COO matrix
    n_cols = 2 + len(all_rubrics) + len(all_features) + 1 # city, rating, rubrics, features, feature_other
    sparse_matrix = coo_matrix((data_values, (row_indices, col_indices)), shape=(n_samples, n_cols))

    return sparse_matrix.tocsr() # Convert to CSR format for efficient operations

# Get all unique rubrics and features from the training data
all_rubrics_train = sorted(list(set(rubric for rubrics_list in clean_data_train['rubrics_id'].str.split().fillna('') for rubric in rubrics_list)))
all_features_train = sorted(list(set(feature for features_list in clean_data_train['features_id'].str.split().fillna('') for feature in features_list)))

# Create sparse matrices for train and test data
sparse_data_train = create_sparse_features(clean_data_train, all_rubrics_train, all_features_train)
sparse_data_test = create_sparse_features(clean_data_test, all_rubrics_train, all_features_train)

print("Sparse matrices created.")
print(f"Shape of sparse_data_train: {sparse_data_train.shape}")
print(f"Shape of sparse_data_test: {sparse_data_test.shape}")

Данные готовы, и теперь можно запустить катбуст

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
!pip install catboost

In [ ]:
# <USE IT!>
clf = CatBoostClassifier()
clf.fit(sparse_data_train, clean_data_train['average_bill'])

In [ ]:
from sklearn.metrics import balanced_accuracy_score

# Make predictions on the test set
catboost_test_pred = clf.predict(sparse_data_test)

# Calculate balanced accuracy on the test set
catboost_balanced_accuracy = balanced_accuracy_score(clean_data_test['average_bill'], catboost_test_pred)

# Print the result rounded to two decimal places
print(f"{catboost_balanced_accuracy:.2f}")

**11. Пришлите в Контест balanced_accuracy_score на тестовой выборке, округлённый до двух знаков после запятой**. Стало ли сильно лучше от того, что мы воспользовались таким крутым классификатором?